In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from __future__ import print_function
np.random.seed(0)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular

In [133]:
menop={"premeno":"no","ge40":'after40',"lt40":"before40"}

#vec={'blabla':'0-2', 'xx':'3.5', 'fwqf':'6-8', 'asgasg':'9-11', 'sagasggas':'12-14', 'asgasgsag':'15-17', 'axcwq':'18-20', 'wqgfasg':'21-23', '24-26':'24-26', 
#     '27-29':'27-29', '30-32':'30-32', '33-35':'33-35', '36-39':'36-39'}
transf={
    2:lambda x:map_array_values(x,menop),
    #4:lambda x:map_array_values(x,vec),
}

In [2]:
menopause = set(["premeno", "ge40", "lt40"])
inv_nodes = set(["0-2", "3-5", "6-8", "9-11", "12-14", "15-17", "18-20", "21-23", "24-26",
                 "27-29", "30-32", "33-35", "36-39"])
def map_array_values(array, value_map):
    # value map must be { src : target }
    ret = array.copy()
    for src, target in value_map.items():
        ret[ret == src] = target
    return ret
def violations_fn(x):
    x = x.astype(float)
    d = np.digitize(x, [0, 5, float('inf')],right=True).astype('|S128')
    return map_array_values(d, {'1': 'low degree', '2': 'medium degree', '3': 'high degree'})
irrad = {
        'yes':'da',
        'no':'nu'
}
transf = {
            2:  lambda x: np.array([y in menopause for y in x]).astype(float),
            4:  lambda x: np.array([y in inv_nodes for y in x]).astype(float),
        }

In [3]:

dataset = utils.load_csv_dataset(os.path.join('/home/ml/pylab','adult/adult.data'),0,',',['Class','Age','Menopause','Tumor-size','Inv-nodes_Lymph-nodes','Node-caps','Degree-malignancy','Breast','Breast-quad','Irradiat']
                                                ,[1,3,5,6,9], [1,2,3,4,5,6,9],transf,True)
#dataset = utils.load_csv_dataset(os.path.join('/home/ml/pylab','adult/adult.data'),0,',',['Class','Age','Menopause','Tumor-size','Inv-nodes','Node-caps','Deg-malig','Breast','Breast-quad','Irradiat']
#                                                ,None, None,None,True,False)


In [319]:
dataset = utils.load_csv_dataset(os.path.join('/home/ml/pylab','adult/adult.data'),0,',',['Class','Age','Menopause','Tumor-size','Inv-nodes','Node-caps','Deg-malig','Breast','Breast-quad','Irradiat']
                                                ,[1,2,3,4,5,6,7,8,9], [1,2,3,4,5,6,7,8,9],transf,True)

In [222]:
dataset = utils.load_csv_dataset(os.path.join('/home/ml/pylab','adult/adult.data'),0,',',['Class','Age','Menopause','Tumor-size','Inv-nodes','Node-caps','Deg-malig','Breast','Breast-quad','Irradiat']
                                                ,[1,2,3,4,5,6,9], [1,2,3,4,5,6,9],transf,True,True)

In [227]:
dataset = utils.load_csv_dataset(os.path.join('/home/ml/pylab','adult/adult.data'),0,',',['Class','Age','Menopause','Tumor-size','Inv-nodes','Node-caps','Deg-malig','Breast','Breast-quad','Irradiat']
                                                ,[1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8,9],feature_transformations=transf)

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

Train 0.9035087719298246
Test 0.7241379310344828


In [20]:
from sklearn.neighbors import KNeighborsClassifier
cl= KNeighborsClassifier(n_neighbors = 4, p = 10)
cl.fit(dataset.train, dataset.labels_train)

KNeighborsClassifier(n_neighbors=4, p=10)

In [5]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [7]:
q=[]
q=[i for i in range(14)]+[i for i in range(271,286)]
dataset.test_idx=np.array(q,dtype='int')#indicii 29

In [8]:
dataset.test_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282,
       283, 284, 285])

In [10]:
dataset.test[28]

array([3., 0., 5., 0., 1., 2., 0.])

In [9]:
for i in range(0,14):
    dataset.test[i]=dataset.data[i]
t=14
for i in range(271,286):
    dataset.test[t]=dataset.data[i]
    t+=1

In [11]:
dataset.data[285]

array([3., 0., 5., 0., 1., 2., 0.])

In [17]:
idx = -1
np.random.seed(0)
print('Prediction: ', explainer.class_names[c.predict(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

Prediction:  b'recurrence-events'


In [23]:
idx = -1
np.random.seed(0)
print('Prediction: ', explainer.class_names[cl.predict(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], cl.predict, threshold=0.95)

Prediction:  b'recurrence-events'


In [24]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: Tumor-size = 30-34 AND Degree-malignancy = 3 AND Age = 50-59 AND Node-caps = no
Precision: 1.00
Coverage: 0.03


In [14]:

# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(dataset.test[:, exp.features()] == dataset.test[idx][exp.features()], axis=1))[0]
print('Anchor test precision: %.2f' % (np.mean(c.predict(dataset.test[fit_anchor]) == c.predict(dataset.test[idx].reshape(1, -1)))))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(dataset.test.shape[0])))

Anchor test precision: 1.00
Anchor test coverage: 0.03


In [ ]:
dataset = utils.load_dataset('adult', balance=True,  dataset_folder='/home/ml/pylab',discretize=True)


In [ ]:

dataset = utils.load_dataset('adult', balance=True,  dataset_folder='/home/ml/pylab',discretize=True)
#dataset = pd.read_csv("breast-cancer.csv",header=None)

In [ ]:
dataset

In [ ]:
dataset.dtypes

In [ ]:
df=dataset.apply(lambda x: x.astype('category'))  

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder# creating initial dataframe
labelencoder = LabelEncoder()# Assigning numerical values and storing in another column


In [ ]:
df1=df.apply(lambda x: labelencoder.fit_transform(x) )

In [ ]:
df1

In [ ]:
df1.hist()

In [ ]:
df1.isnull().sum()
df1.isna().sum()

In [ ]:
X=df1[df1.columns[1:10]]
Y=df1[df1.columns[0]]

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:

ac=[]
#Using Logistic Regression Algorithm to the Training Set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))
ac

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
cl= KNeighborsClassifier(n_neighbors = 4, p = 10)
cl.fit(X_train, Y_train)

Y_pred = cl.predict(X_test)
from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))
ac

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))

In [ ]:
#ac=[]
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, random_state = 0)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
ac.append(accuracy_score(Y_test, Y_pred))

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 14, kernel_initializer = 'uniform', activation = 'relu', input_dim = 9))
    classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier =  build_classifier('rmsprop')

classifier.fit(X_train, Y_train,batch_size = 1, epochs = 20)
Y_pred=(classifier.predict(X_test) > 0.5).astype("int32")
ac.append(accuracy_score(Y_test, Y_pred))

In [ ]:
#validation accuracy
ac

In [ ]:
mv=max(ac)
listMaxIndex=[i for i, j in enumerate(ac) if j == mv]

In [ ]:
algoList=['LogReg','KNN','SVC','GaussNB','DT','RF','NN']

In [ ]:
[algoList[i] for i in listMaxIndex]


In [ ]:
max(ac)

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)